# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
import pandas as pd
import numpy as np
import glob
import ntpath
import os

In [1]:
DATA_FOLDER = '../../Tutorials/02 - Intro to Pandas/Data/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

Potential problems:
- Column name (same for rows) + inconsistent case sensitive
Date,Variable,National
date,variable,...National
Date,Description,Totals

- Split rows
12/1/2014,"Total death/s in confirmed, 
 probable, suspected cases",3161,158,124,10,63,74,5,26,384,549,17,1651,53,8,25,14,
 
- Unnamed/partial columns
- Some rows are counts of others
New cases of suspects, New cases of probables, New cases of confirmed, Total new cases registered so far

In [2]:
import glob
import pandas as pd
import os

countries_paths = glob.glob(DATA_FOLDER + 'ebola/*/')
countries_names = [os.path.basename(os.path.normpath(x)).replace('_data', '').capitalize() for x in countries_paths]

In [3]:
print(countries_names)

['Guinea', 'Liberia', 'Sl']


In [4]:
aliases = {
    'variable': 'description',
    'national': 'totals'
}

In [5]:
dfs = []
for i, path in enumerate(countries_paths):
    filenames = glob.glob(path + "/*.csv")
    
    for f in filenames:
        df = pd.read_csv(f)
        df.columns = [c.lower() for c in df.columns] # Set all columns to lower-case
        df.rename(columns=aliases, inplace=True)
        df['country'] = countries_names[i]
        df['date'] = [parser.parse(x, tzinfos={'dayfirst': True}) for x in df['date']]
        df['totals'] = pd.to_numeric(df['totals'], errors='coerce')
        df = df.fillna(0)
        dfs.append(df[['date', 'country', 'description', 'totals']])
        
df = pd.concat(dfs)
df['description'] = [x.lower() for x in df['description']]
#print(df['description'].unique())

NameError: name 'parser' is not defined

In [ ]:
from itertools import product

column_keywords = {
    'new_cases': ['new cases of suspects', 'new cases of probables', 'new cases of confirmed', 'new case/s (suspected)',
                 'new case/s (probable)', 'new case/s (confirmed)',  'new_suspected', 'new_probable', 'new_confirmed'],
    'new_deaths': ['new deaths registered today', 'new deaths registered', 'newly reported deaths',
                  'etc_new_deaths']
}

desc_map = []
for k, v in column_keywords.items():
    for alias in v:
        desc_map.append((k, alias))
desc_map = pd.DataFrame(desc_map, columns=['type', 'description'])
df = df.merge(desc_map, on='description', how='inner')
df = df.drop('description', axis=1)

In [ ]:
df.head()

In [ ]:
pivoted = df.pivot_table(index=['date', 'country'], columns='type', values='totals', aggfunc='sum')
#pivoted = pivoted['totals'].reset_index()
pivoted.columns.name = None
df = pivoted.reset_index()

In [ ]:
from dateutil import parser
df_ = df.groupby([df.date, df.country, df.type]).sum().reset_index()
df_.head()

In [ ]:
df[df.country == 'Liberia'].head()

In [ ]:
df_ = df.groupby([df.country, df.date.dt.year, df.date.dt.month]).mean()
df_.index.names = ['country', 'year', 'month']
df_.reset_index().sort_values(ascending=[True, True, True], by=['country', 'year', 'month'])

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
path = DATA_FOLDER+"microbiome"
metadata = glob.glob(path + "/metadata.xls")
filenames = glob.glob(path + "/MID*.xls")

metadata = pd.read_excel(metadata[0])

rnaData = []
for i, file in enumerate(filenames):
    currData = pd.read_excel(file, header=None, names=["Taxon", "Count"])
    currData["BARCODE"] = "MID"+str(i+1) #add a column value to do the join
    rnaData.append(currData)

result = pd.concat(rnaData)
result = pd.merge(result, metadata, how='inner', on="BARCODE")

result = result.fillna("unknown")
# there are no NaN values 
print(pd.isnull(result).sum()) 

# set an index that is unique
result.set_index(["Taxon","BARCODE"], inplace=True) # set the first column as index
print(result.index.is_unique)

# print(result.index)
# result["Archaea \"Crenarchaeota\" Thermoprotei Desulfurococcales Desulfurococcaceae Ignisphaera","MID1"]
result.head()

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here